# Python is used to run the tasks in a data pipeline

# [SETUP]

In [ ]:
! python ../setup.py --db_file tpch.db --sqlite_db_file example.db

# Use the right data structure for your data access needs

- most code logic that controls data pipeline will involve one for of control flow
* if..elif..else
* for loop

You will mostly do ne of the following:
1. iterate: operate on one element from a list of elements at a time
2. lookup: quickly access value given a key

### `List` for iteration and `dict` for lookup

1. `Lists`: In Python, lists are ideal for storing a collection of items that you want to iterate over. Lists are ordered, mutable, and can contain duplicate elements.
2. `Dictionaries`: Dictionaries (dict) are perfect for situations where you need fast lookups by key. A dictionary stores key-value pairs and provides average O(1) time complexity for lookups.

In [ ]:
# List for Iteration
names = ["Alice", "Bob", "Charlie"]
for name in names:
    print(f"Hello, {name}!")

In [ ]:
# Dict for Lookup
age_lookup = {
    "Alice": 30,
    "Bob": 25,
    "Charlie": 35
}
print(f"Alice's age is {age_lookup['Alice']}")  # Fast lookup by key

### Functions allow you to reuse blocks of code

A function is a block of code that can be re-used as needed. This allows for us to have logic defined in one place, making it easy to maintain and use.

Often referred to as DRY (dont repeat yourself)

In [ ]:
def gt_three(input_list):
    result = []
    for elt in input_list:
        if elt > 3:
            result.append(elt)
    return result

In [ ]:
gt_three([1,2,3,4,5,6])

### Define a blueprint with a `Class` and create `Objects` from it

Think of a class as a blueprint and objects as things created based on that blueprint

In data pipelines we generally create a base class to represent "how" your data pieline should work (not what transformation it does).
When creating pipelines you'd generally inherit from the base class and make necessary changes. 

add: inheritance image

In [ ]:
class DataExtractor:

    def __init__(self, extractor_id):
        self.extractor_id = extractor_id

    def get_connection(self):
        print(f'Getting {self.extractor_id}s connection')
        return
        # Some logic

    def close_connection(self):
        print(f'Closing {self.extractor_id}s connection')
        # Some logic

In [ ]:
csv_data_extractor = DataExtractor("csv")
csv_data_extractor.get_connection()

In [ ]:
json_data_extractor = DataExtractor("json")
json_data_extractor.get_connection()

**Inheritance**

- base class: define functions and its arguments
- child class: implement function specific to its use case

In [ ]:
import os
from abc import ABC, abstractmethod # python module to define abstract interfaces

# Abstract class with abstract methods
class SocialETL(ABC):
    @abstractmethod
    def extract(self, id, num_records, client):
        pass

    @abstractmethod
    def transform(self, social_data):
        pass

    @abstractmethod
    def load(self, social_data, db_conn):
        pass

    @abstractmethod
    def run(self, db_conn, client, id, num_records):
        pass

# Concrete implementation of the abstract Class
class RedditETL(SocialETL):
    def extract(self, id, num_records, client):
        # code to extract reddit data
        print("Reddit extract")

    def transform(self, social_data):
        # code to transform reddit data
        print("Reddit transform")

    def load(self, social_data, db_conn):
        # code to load reddit data into the final table
        print("Reddit load")

    def run(self, db_conn, client, id, num_records):
        # code to run extract, transform and load
        print("Reddit ETL run")

# Concrete implementation of the abstract Class
class TwitterETL(SocialETL):
    def extract(self, id, num_records, client):
        # code to extract reddit data
        print("Twitter extract")

    def transform(self, social_data):
        # code to transform reddit data
        print("Twitter transform")

    def load(self, social_data, db_conn):
        # code to load reddit data into the final table
        print("Twitter load")

    def run(self, db_conn, client, id, num_records):
        # code to run extract, transform and load
        print("Twitter ETL run")

# This "factory" will acccept an input and give you the appropriate object that you can use to perform ETL
def etl_factory(source):
    factory = {
        'Reddit': RedditETL(),
        'Twitter': TwitterETL()
    }
    if source in factory:
        return factory[source]
    else:
        raise ValueError(
            f"source {source} is not supported. Please pass a valid source."
        )

# calling code
source = 'Reddit'
social_etl = etl_factory(source)
social_etl.run(db_conn='fake_db_conn', id='fake_id', num_records=100, client='fake_client')

# Python can push data to/pull data from any system

In data pipelines, Python is used as the glue to move data between systems. Python can orchestrate movement of data across sytems in one of 2 main ways

1. PUll data into the Python process and push it to the destination. This approach requires that the data pulled into process meaning that the data size should be handle-able by the Python process
2. Instruct other systems to move/transform data. In this approach Python acts as an orchestrator telling the other systems what to do. 

add: image and details Data is stored on disk and processed in memory

## Interact with databases using their specific Python packages 

Python is a very popular language ad:tiobe link, and as such most database engines have python libraries specifically designed to interact with them.

e.g. sqlite3, postgres, mysql, snowflake, redshift, duckdb

In data pipelines, this is typically use to instruct your database engine to transform data 
* pull data and push it into a different systems (some database engines have native feature to push data to cloud storage, e.g. snowflake -> S3)

In [ ]:
import sqlite3

# Connect to an SQLite database (or create it)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Query data (pull)
cursor.execute('SELECT * FROM users') # We can run any SQL query here
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
conn.close()

## Interact with API endpoint using the `requests` package

In data pipelines you may have to interact (usually pull data from) with APIs. 

While there are many libraries to interact with APIs the most popular one is `requests`

When pulling data from an API, remember to 
* paginate: api data pulls can only send back a limited set of data at a time (due to bandwith constraints). So you will usually have to make multiple calls to the same API.
* query params: Most apis enable you to ask it for specific segments of data, you can do this using query params. Do this with add: feature
* rate limiting: The API is usually powered by >=1 servers, and if you repeated call the API multiple times without any breaks you may overwhelm the server (DOS attack). To prevent this and for performance most APIs have a limit on the number of API calls you can perform a minute. Control with add: feature 

In [ ]:
import requests

# Pull data (GET request)
response = requests.get('https://jsonplaceholder.typicode.com/posts')
data = response.json()

# Print the first post
print(f'Data pulled: {data[0]}')

# Push data (POST request)
new_post = {
    'title': 'New Post',
    'body': 'This is the content of the new post.',
    'userId': 1
}
response = requests.post('https://jsonplaceholder.typicode.com/posts', json=new_post)
print(f'Data posted: {response.json()}')


## Interact with files in your filesystem with Python's standard libraries

Python can read write to files of most formats csv, xml, json, parquet, etc

In [ ]:
import csv

# Write CSV to a local file
data = [["Name", "Age"], ["Alice", 30], ["Bob", 25]]
filename = "sample.csv"
with open(filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [ ]:
! cat ./sample.csv

In [ ]:
import os

# Delete the file if it exists
if os.path.exists(filename):
    os.remove(filename)

# Run SQL queries using Python

Python can be used to run SQL queries to transform/ load data. 

Without Python, you'd need a system (like dbt which is itself a python library) to run your SQL queries.

In [ ]:
import sqlite3

# Connect to an SQLite database (or create it)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Query data (pull)
cursor.execute('SELECT * FROM users') # We can run any SQL query here
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('Chester', 9000))
cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('Geppato', 50))
# conn.commit() # Uncomment this line, else the insert will not be committed into your databsae

# Query data (pull)
cursor.execute('SELECT id, count(*) FROM users GROUP BY id ORDER BY id') # We can run any SQL query here
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
conn.close()

In [ ]:
import sqlite3

# Connect to an SQLite database (or create it)
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Query data (pull)
cursor.execute('SELECT * FROM users') # We can run any SQL query here
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

# Dataframes provides a Pythonic way to transform data

Python popularized dataframe based data transformations with Pandas, which was a result of dataframe popularity in R for data science.

Dataframe allows us to load data into memory (Pandas) and perform transformations. Most operations that can be done in SQL can be done in Dataframe.

Lates tech like Polars can handle data that is larger in size than your python process memory.

In [ ]:
import duckdb

db_file_name = './tpch.db'
conn = duckdb.connect(db_file_name)
cursor = conn.cursor()

# Connect to DuckDB and load TPC-H tables into Pandas DataFrames
customer_df = cursor.sql("SELECT * FROM customer").df()
orders_df = cursor.sql("SELECT * FROM orders").df()
lineitem_df = cursor.sql("SELECT * FROM lineitem").df()
nation_df = cursor.sql("SELECT * FROM nation").df()
region_df = cursor.sql("SELECT * FROM region").df()
supplier_df = cursor.sql("SELECT * FROM supplier").df()
part_df = cursor.sql("SELECT * FROM part").df()
partsupp_df = cursor.sql("SELECT * FROM partsupp").df()

conn.close()

In [ ]:
import pandas as pd

In [ ]:
# Assuming 'customer_df' is the DataFrame containing the customer table data
filtered_df = customer_df[customer_df["c_nationkey"] == 20].head(10)
filtered_df

In [ ]:
customer_df[
    ((customer_df["c_nationkey"] == 20) & (customer_df["c_acctbal"] > 1000)) |
    (customer_df["c_nationkey"] == 11)
].head(10)

In [ ]:
# Inner join
inner_join_df = orders_df.merge(
    lineitem_df,
    left_on="o_orderkey",
    right_on="l_orderkey",
    how="inner"
)

inner_join_df[
    (inner_join_df["o_orderdate"] >= inner_join_df["l_shipdate"] - pd.Timedelta(days=5)) &
    (inner_join_df["o_orderdate"] <= inner_join_df["l_shipdate"] + pd.Timedelta(days=5))
].head(2)

In [ ]:
# Left join
left_join_df = orders_df.merge(
    lineitem_df,
    left_on="o_orderkey",
    right_on="l_orderkey",
    how="left"
)

left_join_df[
    (left_join_df["o_orderdate"] >= left_join_df["l_shipdate"] - pd.Timedelta(days=5)) &
    (left_join_df["o_orderdate"] <= left_join_df["l_shipdate"] + pd.Timedelta(days=5))
].head(2)

In [ ]:
# Right join
right_join_df = orders_df.merge(
    lineitem_df,
    left_on="o_orderkey",
    right_on="l_orderkey",
    how="right"
)

right_join_df[
    (right_join_df["o_orderdate"] >= right_join_df["l_shipdate"] - pd.Timedelta(days=5)) &
    (right_join_df["o_orderdate"] <= right_join_df["l_shipdate"] + pd.Timedelta(days=5))
].head(2)

In [ ]:
# Full join
full_join_df = orders_df.merge(
    lineitem_df,
    left_on="o_orderkey",
    right_on="l_orderkey",
    how="outer"
)

full_join_df[
    (full_join_df["o_orderdate"] >= full_join_df["l_shipdate"] - pd.Timedelta(days=5)) &
    (full_join_df["o_orderdate"] <= full_join_df["l_shipdate"] + pd.Timedelta(days=5))
].head(2)